## 03 Predictions & bootstrapped condfidence intervals

In [2]:
# import libraries
import numpy as np
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt

### Apply the model to the PBS data to get predictions

In [ ]:
# best model taken from optimisation in previous steps
best_model = RandomForestClassifier(criterion='entropy', max_depth=10, 
                    max_features=15, n_estimators=500, random_state=0)

# declare feature vector and target variable in training set
X = rt_data.drop(['reinfection'], axis=1)
y = rt_data['reinfection']
best_model.fit(X, y)

# use best model after comparing output of nested cross validations 
#for random forest and gradient boosting classifiers
prediction = best_model.predict(pbs_data)

### Compute the bootstrapped confidence intervals

In [ ]:
# configure bootstrap
n_iterations = 1000
 
# run bootstrap
predictions_list = list()
for i in range(n_iterations):
    # prepare train and test sets
    train = rt_data.sample(frac=1, replace=True)
    # fit model
    X = train.drop(['reinfection'], axis=1)
    y = train['reinfection']
    best_model.fit(X, y)
    # predict on PBS
    prediction = best_model.predict(pbs_data) 
                                              
    # store predictions
    predictions_list.append(prediction)

# transform into array
predictions_array = np.vstack(predictions_list) 

## to get number and proprotion of predictions for reinfection
num_reinfections = predictions_array.sum(axis=1)
mean_reinfections = predictions_array.mean(axis=1)

## to get number and proprotion of predictions for virological failure
num_vf = (predictions_array==0).sum(axis=1)
mean_vf = (predictions_array==0).mean(axis=1)

In [ ]:
# plot scores
sns.set(rc = {'figure.figsize':(7,5)})
plt.hist(num_reinfections, bins=30, color='darkturquoise', alpha=0.6)
plt.ylabel('Number of models', fontsize=14)
plt.xlabel('Number of retreatments for reinfection', fontsize=14)
plt.tight_layout()
plt.savefig('Figures/bootstrapCI_github_absolute.png')
plt.show()

# plot prediction scores for reinfection retreatment as proportion
sns.set(rc = {'figure.figsize':(7,5)})
plt.hist(mean_reinfections, bins=30, color='darkturquoise', alpha=0.6)
plt.ylabel('Number of models', fontsize=14)
plt.xlabel('Proportion of retreatments for reinfection', fontsize=14)
plt.tight_layout()
plt.savefig('Figures/bootstrapCI_github_proportion.png')
plt.show()